In [10]:
import pandas as pd
import itertools
from constants import *
from functions import bin_helper
import numpy as np
import matplotlib.pyplot as plt 

In [11]:
df_train = pd.read_csv("split/train.csv")

In [13]:
mutual_fund_df = {}

for ((i, xchg), (j, stk)) in itertools.product(enumerate(exchanges), enumerate(stocks)):
    df = df_train[(df_train.xchg == xchg) & (df_train.stk == stk) & (df_train.cp == "MF")]

    if stk == "D":
        df = df[df.sz != 8]

    n = len(df)
    A = np.vstack([
        df.pred_edge,
        df.px,
        df.sz,
        # df.sm,
        np.ones((1, n))]).T

    ys = (df.real_edge - df.pred_edge).values.reshape((n, -1))

    x0 = np.linalg.lstsq(A, ys, rcond=None)[0].flatten()
    mutual_fund_df[(xchg, stk)] = {
        'pred_edge_slope': x0[0],
        'px_slope': x0[1],
        'sz_slope': x0[2],
        # 'sm_slope': x0[3],
        'offset': x0[3]
    }

# mutual_fund_df.to_csv('mutual_fund_lines.csv')
mutual_fund_df = pd.DataFrame.from_dict(mutual_fund_df, orient='index')
mutual_fund_df.index = mutual_fund_df.index.rename(['xchg', 'stk'])
mutual_fund_df.to_csv("mutual_fund_lines.csv")

IndexError: index 4 is out of bounds for axis 0 with size 4